In [2]:
import pandas as pd
import openpyxl
import h5py
import cv2
import numpy as np
import os
import sys
sys.path.append('C:/Users/w37262do/Documents/git/PyIR/src')
import spectral_preprocessing as sp
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter

## Define utility functions

In [3]:
master = pd.read_excel(r'D:/datasets/pcuk2023_ftir_whole_core/master_sheet.xlsx')

slides = master['slide'].to_numpy()
core_ids = master['core_id'].to_numpy()
patient_ids = master['patient_id'].to_numpy()
t_or_ns = master['t_or_n'].to_numpy()
pathology_infos =master['pathology_info'].to_numpy()
annotated_pixels =master['annotated_pixels'].to_numpy()
hdf5_filepaths = master['hdf5_filepath'].to_numpy()
annotation_filepaths = master['annotation_filepath'].to_numpy()
chemical_image_filepaths = master['chemical_image_filepath'].to_numpy()
mask_filepaths = master['mask_filepath'].to_numpy()

In [4]:
patch_width = patch_height = pw = ph = 25
annotation_class_colors = np.array([[0,255,0],[128,0,128],[255,0,255],[0,0,255],[255,165,0],[255,0,0],[0,255,255],[255,255,0],])#[127,0,0]])
annotation_class_names = np.array(['epithelium_n','stroma_n','epithelium_c','stroma_c','corpora_amylacea','blood',"crushed","immune_infiltrate"])#,

In [5]:
def process_core_from_path(idx):
    hf = h5py.File(hdf5_filepaths[idx],'r')
    spectra = hf['spectra'][:]
    hf.close()
    mask = cv2.imread(mask_filepaths[idx])[:,:,1]
    annotations = cv2.imread(annotation_filepaths[idx])[:,:,::-1]
    chemical = cv2.imread(chemical_image_filepaths[idx])[:,:,1]
    
    annotation_mask = np.zeros((*mask.shape,len(annotation_class_names)))
    for tissue_class in range(len(annotation_class_names)):
        annotation_mask[:,:,tissue_class] = np.all(annotations == annotation_class_colors[tissue_class].reshape(1,1,-1),axis=-1)
    
    return spectra,annotations,annotation_mask,chemical,mask

## Define dataset parameters

In [6]:
root_dir = r'D:/datasets/pcuk2023_ftir_25px'

new_sheet_data = {
    'slide':[],
    'core_id':[],
    'patient_id':[],
    'origin_x':[],
    'origin_y':[],
    'tissue_class':[],
    't_or_n':[],
    'pathology_info':[],
    'annotated_pixels_class':[],
    'annotated_pixels_total':[],
    'hdf5_filepath':[],
    'annotation_filepath':[],
    'chemical_image_filepath':[],
    'mask_filepath':[],
}
for core_idx in range(len(hdf5_filepaths)):
    print(f"Processing cores, on core: {core_idx+1}/{len(hdf5_filepaths)}, tissue type: ",end="")
    s = slides[core_idx]
    c = core_ids[core_idx]
    p = patient_ids[core_idx]
    tn = t_or_ns[core_idx]
    path_info = pathology_infos[core_idx]
    
    # load in core, annotations, etc
    spectra,annotations,annotation_mask,chemical,mask = process_core_from_path(core_idx)
    
    # make gaussian blurred annot masks
    annotation_mask_gaussian = np.zeros_like(annotation_mask)
    for tissue_class in range(len(annotation_class_names)):
        annotation_mask_gaussian[:,:,tissue_class] = gaussian_filter(annotation_mask[:,:,tissue_class], sigma=1,radius=12)
        
    # for each tissue class
    for tissue_class in range(len(annotation_class_names)):
        print(f"{tissue_class}, ",end="")
        # skip classes with no annotations
        if annotation_mask[:,:,tissue_class].sum() == 0: continue
        
        # make binary mask for taken pixels
        taken_mask = np.zeros_like(annotation_mask_gaussian[:,:,tissue_class])
        # sort pixels from most to least annotations nearby
        sorted_pixels = np.stack([
            np.argsort(annotation_mask_gaussian[:,:,tissue_class],axis=None)//256,
            np.argsort(annotation_mask_gaussian[:,:,tissue_class],axis=None)%256,
        ],axis=1)[::-1]
        # for each pixel, from most to least annotated
        for px in sorted_pixels:
            
            # if centre pixel is not class, skip
            if annotation_mask[*px,tissue_class] == 0: continue
            # if 'current best' is 0, no more pixels worth looking at
            if annotation_mask[*px,tissue_class] == 0: break
            
            # take patch
            u = px[0]-ph//2
            d = 1+px[0]+ph//2
            l = px[1]-pw//2
            r = 1+px[1]+pw//2
            if u < 0 or l < 0 or r > 255 or d > 255: continue # patch don't fit
            patch_spectra = spectra[u:d,l:r,:]
            patch_taken = taken_mask[u:d,l:r,]
            patch_chemical = chemical[u:d,l:r,]
            patch_annotations = annotations[u:d,l:r,:]
            patch_annotations_mask = annotation_mask[u:d,l:r,:]
            patch_mask = mask[u:d,l:r,]
            
            # Skip if patch is mostly contained in another patch
            if patch_taken.sum() > (ph*pw)/2: continue
            
            # take the patch and save it
            patchname = f's{s:0>{2}}_c{c:0>{3}}_x{px[1]:0>{3}}_y{px[0]:0>{3}}'
            spectral_savepath = f'{root_dir}/spectral/{patchname}.h5'
            chemical_savepath = f'{root_dir}/chemical/{patchname}.png'
            mask_savepath = f'{root_dir}/mask/{patchname}.png'
            annotation_savepath = f'{root_dir}/annotation/{patchname}.png'
            
            hf = h5py.File(spectral_savepath, 'w')
            hf.create_dataset('spectra', data=patch_spectra, compression='lzf')
            hf.close()
            cv2.imwrite(chemical_savepath, patch_chemical.astype(np.uint8))
            cv2.imwrite(annotation_savepath, patch_annotations.astype(np.uint8)[:,:,::-1])
            cv2.imwrite(mask_savepath, patch_mask)
            
            # append to mega lists
            new_sheet_data['slide'].append(s)
            new_sheet_data['core_id'].append(c)
            new_sheet_data['patient_id'].append(p)
            new_sheet_data['origin_x'].append(px[1])
            new_sheet_data['origin_y'].append(px[0])
            new_sheet_data['tissue_class'].append(tissue_class)
            new_sheet_data['t_or_n'].append(tn)
            new_sheet_data['pathology_info'].append(path_info)
            new_sheet_data['annotated_pixels_class'].append(patch_annotations_mask[:,:,tissue_class].sum())
            new_sheet_data['annotated_pixels_total'].append(patch_annotations_mask[:,:,:].sum())
            new_sheet_data['hdf5_filepath'].append(spectral_savepath)
            new_sheet_data['annotation_filepath'].append(annotation_savepath)
            new_sheet_data['chemical_image_filepath'].append(chemical_savepath)
            new_sheet_data['mask_filepath'].append(mask_savepath)
        
            # fill the taken map
            taken_mask[u:d,l:r,] = 1
    print("")
    

Processing cores, on core: 1/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 2/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 3/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 4/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 5/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 6/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 7/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 8/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 9/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 10/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 11/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 12/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 13/235, tissue type: 0, 1, 2, 3, 4, 5, 6, 7, 
Processing cores, on core: 14/235, tissue type: 0, 1, 2, 3, 

In [7]:
df = pd.DataFrame.from_dict(new_sheet_data)
df.to_excel(fr"{root_dir}/master_sheet.xlsx",index=False)

In [8]:
np.unique(np.array(new_sheet_data['tissue_class']),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([1845, 1846, 2202, 1221,  154,  197,   24,   70], dtype=int64))